### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6489,2023-11-22,Brasil Nbb,19:00,Paulistano,Pato,1.28,3.59,145.5,1.88,1.88,-9.5,2.08,1.67,CMtEEOfT,0.781250,0.278552,0.531915,0.531915,0.059802,0.0,0.4,NaN,NaN,140.782,26.236601,0.186363,285.100,147.631324,0.517823,0.000,187.740,150.75,271.35,0.0,0.0,0.0,0.0,0.670808,0.000000,0.154621,0.00,0.00,inf,0.000000,0.0,0.000000,-1.18,-0.236,-10.974576,0.000000,0.0,0.000000
6490,2023-11-22,Brasil Nbb,19:30,Bauru,Caxias do Sul,1.24,3.94,148.5,1.82,1.84,-10.5,2.04,1.69,hGOtAr3p,0.806452,0.253807,0.549451,0.543478,0.060259,0.4,0.0,NaN,NaN,144.996,67.718639,0.467038,149.624,28.310590,0.189212,126.796,0.000,121.60,175.56,0.0,0.0,0.0,0.0,0.737138,0.007728,0.132701,2.70,0.54,0.444444,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6491,2023-11-22,Brasil Nbb,20:00,Franca,União Corinthians,1.04,8.80,162.5,1.80,1.86,-17.5,2.02,1.70,W6Np92Ij,0.961538,0.113636,0.555556,0.537634,0.075175,0.0,0.0,NaN,NaN,114.258,33.042236,0.289190,161.900,62.889022,0.388444,0.000,0.000,99.45,240.53,0.0,0.0,0.0,0.0,1.115274,0.023184,0.121653,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6492,2023-11-22,Eua Nba,00:00,Los Angeles Lakers,Utah Jazz,1.36,3.61,237.5,1.89,2.04,-7.5,1.97,1.97,WQodBj3n,0.735294,0.277008,0.529101,0.490196,0.012302,0.8,0.6,NaN,NaN,170.072,32.586840,0.191606,322.484,88.971003,0.275893,180.380,541.022,153.30,331.47,0.0,0.0,0.0,0.0,0.640238,0.053978,0.000000,0.65,0.13,2.769231,0.653202,0.7,0.046798,-2.39,-0.478,-5.460251,0.284729,0.2,-0.084729
6493,2023-11-22,Eua Nba,21:00,Charlotte Hornets,Washington Wizards,1.61,2.53,239.5,1.81,2.07,-4.5,2.05,1.88,djWQBCpU,0.621118,0.395257,0.552486,0.483092,0.016375,0.6,0.6,NaN,NaN,307.792,119.627454,0.388663,423.032,114.919031,0.271656,259.402,411.764,353.16,551.05,1.0,1.0,0.0,0.0,0.314270,0.094767,0.061175,-2.60,-0.52,-1.173077,0.319144,0.2,-0.119144,-2.56,-0.512,-2.988281,0.287307,0.1,-0.187307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6645,2023-11-22,Rússia Superliga,09:30,Novosibirsk,M. Khimki,2.53,1.48,151.5,1.85,1.85,1.5,2.08,1.63,lO5tjcMK,0.395257,0.675676,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,119.28,81.62,0.0,0.0,0.0,1.0,0.370305,0.000000,0.171535,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6646,2023-11-22,Rússia Superliga,10:00,Temp Sumz Revda,Dome-Springs Izhevsk,1.01,9.70,162.5,1.85,1.85,-20.5,1.97,1.71,U7nbTZE7,0.990099,0.103093,0.540541,0.540541,0.093192,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,94.17,0.00,0.0,0.0,0.0,0.0,1.147480,0.000000,0.099917,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6647,2023-11-22,Rússia Superliga,11:00,Chelyabinsk,MBA Moscow 2,1.48,2.53,151.5,1.85,1.85,-5.5,1.97,1.73,rBzthJi8,0.675676,0.395257,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,213.12,0.00,0.0,0.0,0.0,0.0,0.370305,0.000000,0.091733,0.00,0.00,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6648,2023-11-22,Rússia Superliga,13:00,CSKA Moscow 2,Bars Rostov,1.50,2.47,152.5,1.85,1.85,-6.5,1.95,1.74,KCqPsZT0,0.666667,0.404858,0.540541,0.540541,0.071525,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,132.66,207.90,0.0,0.0,0.0,0.0,0.345538,0.000000,0.080484,0.00,0.00,inf,0.000000,0.0,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Los Angeles Lakers,Utah Jazz,237.5,1.89,1.0000,Over
4,21:00,Eua Nba,Charlotte Hornets,Washington Wizards,239.5,1.81,0.9999,Over
6,21:30,Eua Nba,Atlanta Hawks,Brooklyn Nets,231.5,1.87,1.0000,Over
7,21:30,Eua Nba,Boston Celtics,Milwaukee Bucks,233.5,1.86,1.0000,Over
8,22:00,Eua Nba,Houston Rockets,Memphis Grizzlies,211.5,1.82,1.0000,Over
9,22:00,Eua Nba,Minnesota Timberwolves,Philadelphia 76ers,218.5,1.80,1.0000,Over
10,22:00,Eua Nba,New Orleans Pelicans,Sacramento Kings,230.5,1.88,0.9989,Over
11,22:00,Eua Nba,Oklahoma City Thunder,Chicago Bulls,222.5,1.86,1.0000,Over
12,22:00,Eua Nba,San Antonio Spurs,Los Angeles Clippers,228.5,1.80,1.0000,Over
148,22:30,Eua Ncaa,Fresno State,James Madison,151.5,1.91,0.8425,Over
